In [ ]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.linalg import inv
from numpy.random import multivariate_normal
from notebook.services.config import ConfigManager
from traitlets.config.manager import BaseJSONConfigManager
import matplotlib as mpl


# set jupyter configurations
%matplotlib inline
%config InlineBackend.figure_format='retina'
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*1.2

# the commented out configuration code below enables scrolling in slide mode
# TODO figure out a clean way to do this
# path = "/home/stefan/.jupyter/nbconfig" # update with your path
# cm = BaseJSONConfigManager(config_dir=path)
# cm.update('livereveal', {
#               'theme': 'white',
#               'transition': 'fade',
#               'start_slideshow_at': 'selected',
#                 'center' : False,
#                 'width' : 1024,
#                 'height': 768,   
#                 'scroll': True,
#                 'slideNumber': True,
# })

# set global variables
gt_color = 'r'
data_color = '#0000FF'
linreg_color = '#519872'
sbi_color  = '#DE1A1A'

domain = np.linspace(0,1,11)
SIGMA = 0.3

In [ ]:
def quadratic(intercept, gain, curvature, sample=False):
    """ Generate a quadratic polynomial for use with interactive widget. """
    theta = np.array([intercept, gain, curvature])
    y = torch.Tensor(np.array([domain**0, domain**1, domain**2]).T) @ theta
    samples = y + np.random.randn(domain.shape[0]) * SIGMA
    
    plt.plot(domain, y,label=f'Ground truth: $f_x={theta[0]} + {theta[1]}x + {theta[2]}x^2$',color=gt_color)
    if sample:
        plt.scatter(domain, samples, label='Samples', marker='x', color=data_color)
    plt.ylim(-1.2, 1.2)
    plt.xlabel('$x$')
    plt.ylabel('$f_x$')
    plt.legend()
    plt.show()
    
    return {
        'param':[intercept, gain, curvature],
        'y': y,
        'samples':samples
    }

- Ground truth function with unknown parameters
- Sampling is only possible with measurement noise $\sigma^2=0.3$
- Let's have a look at the function

In [ ]:
ground_truth = interactive(quadratic, intercept=(-1.,1.), gain=(-0.2, 0.2), curvature=(-2.,2.), sample=False, plot=True)
ground_truth 

- We desperately want to know the parameters of the original function
- It looks suspiciously like a quadratic polynomal $f(x) = ax^2 + bx + c$, so let's assume it is
- But we need to find out the weights $a$, $b$, and $c$
- How could we find those out?

🛠️ Idea for a practical: Use any method you like to find out parameters and see who gets the closest 🛠️

- Let's try linear regression

In [ ]:
# get data from ground truth
ground_truth_param = np.array(ground_truth.result['param'])
data = ground_truth.result['samples']

# linear regression baseline
features = np.array([domain**0, domain**1, domain**2]).T
linreg_param = inv(features.T @ features).dot(features.T @ data.numpy())
linreg_reconstruction = features @ linreg_param
# and compute error
precicion_matrix = np.array([[(features[:,i]*features[:,j]).sum()/SIGMA**2 for i in range(3)] for j in range(3)])
linreg_covariance = inv(precicion_matrix)
linreg_error = np.diag(linreg_covariance)**0.5

In [ ]:
# plotting
plt.plot(domain, linreg_reconstruction, 'k', label = 'Linear regression', color=linreg_color)
plt.scatter(domain, data, marker='x', label='Samples', color=data_color)
plt.plot(domain,ground_truth.result['y'], 'r:',color=gt_color, alpha=0.3, label='Ground truth')
plt.legend()
plt.ylim(-1.2, 1.2)
plt.xlabel('$x$')
plt.ylabel('$f_x$')
plt.show()

- But this is a Simulation-based Inference workshop
- Couldn't we also build a simulator that models interesting polynomials like this? 

In [ ]:
def quadratic_simulator(param, domain = np.linspace(0, 1, 11), noise_std=SIGMA):
    """ Given parameters, return a quadratic polynomial. """
    transformed_domain = torch.Tensor(np.array([domain**0, domain**1, domain**2]).T)
    noise = torch.randn(domain.shape[0]) * noise_std
    
    return transformed_domain @ torch.Tensor(param) + noise



- How can we use this simulator? 
- Simply try out different parameter combinations until we find a one that produces a result similar to our data?
- Or, we could use the `sbi` toolbox.

In [ ]:
# perform simulation based inference using a uniform prior and our simulator as likelihood
prior = utils.BoxUniform(low=-2*torch.ones(3), high=2*torch.ones(3))
posterior = infer(quadratic_simulator, prior, method='SNPE', num_simulations=300)

In [ ]:
# sample from our posterior over parameters
sample = posterior.sample((10000,), x=data)
sbi_param = sample.mean(dim =0).numpy()
sbi_reconstruction = features @ sbi_param

# plotting
plt.plot(domain, sbi_reconstruction, 'k', label = 'SBI', color=sbi_color)
plt.plot(domain, linreg_reconstruction, label='Linear Regression', color=linreg_color)
plt.scatter(domain, data, marker='x', label='Samples', color=data_color)
plt.plot(domain,ground_truth.result['y'], 'r:',color=gt_color, alpha=0.3, label='Ground truth')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$f_x$')
plt.ylim(-1.2, 1.2)
plt.show()

Let's have a closer look at the parameters predicted by linear regression and SBI respectively.

In [ ]:
# linreg
linreg_samples = multivariate_normal(linreg_param, linreg_covariance, size=10000)
fig, axes = analysis.pairplot(linreg_samples,
                           points=ground_truth_param,
                           points_offdiag={'markersize': 12},
                           points_colors='r',
                           title='Linear Regression',
                           figsize=(6,6))


# sbi
_ = analysis.pairplot(sample,
                      points=ground_truth_param,
                      points_offdiag={'markersize': 12},
                      points_colors='r',
                      title='SBI',
                      figsize=(6,6))